# youtube comment sentiment analysis

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from concurrent.futures import ThreadPoolExecutor, as_completed
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm
import warnings
import scipy
from scipy.stats import t
from sklearn.utils import resample
from datetime import datetime
import sys
import os

warnings.filterwarnings(action="ignore")

MAX_COMMENTS = 10000

sys.path.insert(0, "/Users/tonymeissner/source/CancelCultureImpact/src/")
sys.path.insert(0, "/Users/tonymeissner/source/CancelCultureImpact/src/analysis/")
from config import *
from analyzer_functions import *

tqdm.pandas()

# data preparation

## read data

In [12]:
kanye_comment = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "kanye_west_youtube_comments.csv")
)
manson_comment = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "marilyn_manson_youtube_comments.csv")
)
kelly_comment = pd.read_csv(os.path.join(RAW_DATA_PATH, "r_kelly_youtube_comments.csv"))
seungri_comment = pd.read_csv(
    os.path.join(RAW_DATA_PATH, "seungri_youtube_comments.csv")
)

## formatting

In [13]:
# delete empty comments
kanye_comment = kanye_comment[kanye_comment["text"].notnull()]
manson_comment = manson_comment[manson_comment["text"].notnull()]
kelly_comment = kelly_comment[kelly_comment["text"].notnull()]
seungri_comment = seungri_comment[seungri_comment["text"].notnull()]

In [37]:
def sort_comment_by_date(df):
    df["updateDt"] = pd.to_datetime(df["updateDt"])
    if df["updateDt"].dt.tz is None:
        df["updateDt"] = df["updateDt"].dt.tz_localize('UTC')
    else:
        df["updateDt"] = df["updateDt"].dt.tz_convert('UTC')
    df.sort_values(by="updateDt", ascending=False, inplace=True, ignore_index=True)
    return df

In [15]:
kanye_comment = sort_comment_by_date(kanye_comment)
manson_comment = sort_comment_by_date(manson_comment)
kelly_comment = sort_comment_by_date(kelly_comment)
seungri_comment = sort_comment_by_date(seungri_comment)

In [16]:
# random sampling comment data (max comment data is 10000)
def balance_dataset(df, artist_name):
    # Specify the date to split the data
    split_date = CELEBRITIES[CELEBRITIES["name"] == artist_name]["cancellation_date"].iloc[0]
    split_date = pd.to_datetime(split_date, utc=True)

    print(f"{artist_name}'s canceled date: {split_date}")

    # Split the data into before and after the specified date
    df_before = df[df["updateDt"] < split_date]
    df_after = df[df["updateDt"] >= split_date]

    print(
        f"{artist_name}'s Before canceled date data: {len(df_before)}, After canceled date data: {len(df_after)}"
    )

    # Determine the number of samples needed to balance the dataset
    n_samples = min(len(df_before), len(df_after))

    # set max comments
    if n_samples > MAX_COMMENTS:
        n_samples = MAX_COMMENTS

    # Downsample the larger dataset to match the smaller dataset
    df_before_downsampled = resample(
        df_before, replace=False, n_samples=n_samples, random_state=42
    )
    df_after_downsampled = resample(
        df_after, replace=False, n_samples=n_samples, random_state=42
    )

    print(
        f"{artist_name}'s Before canceled date data: {len(df_before_downsampled)}, After canceled date data: {len(df_after_downsampled)}\n"
    )

    # Combine the downsampled data
    df_balanced = pd.concat([df_before_downsampled, df_after_downsampled])

    return df_balanced

In [17]:
kanye_comment = balance_dataset(kanye_comment, "kanye_west")
manson_comment = balance_dataset(manson_comment, "marilyn_manson")
kelly_comment = balance_dataset(kelly_comment, "r_kelly")
seungri_comment = balance_dataset(seungri_comment, "seungri")

kanye_west's canceled date: 2022-10-07 00:00:00+00:00
kanye_west's Before canceled date data: 59083, After canceled date data: 125303
kanye_west's Before canceled date data: 10000, After canceled date data: 10000

marilyn_manson's canceled date: 2021-02-21 00:00:00+00:00
marilyn_manson's Before canceled date data: 12583, After canceled date data: 42822
marilyn_manson's Before canceled date data: 10000, After canceled date data: 10000

r_kelly's canceled date: 2021-09-27 00:00:00+00:00
r_kelly's Before canceled date data: 27978, After canceled date data: 79477
r_kelly's Before canceled date data: 10000, After canceled date data: 10000

seungri's canceled date: 2019-01-31 00:00:00+00:00
seungri's Before canceled date data: 3181, After canceled date data: 33876
seungri's Before canceled date data: 3181, After canceled date data: 3181



# preprocessing

## emoji

In [18]:
def remove_emojis_parallel(df):
    """
    Remove emojis in parallel from a DataFrame.
    """
    # Initialize the 'remove_emoji' column with NaN
    df["remove_emoji"] = pd.NA

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(remove_emojis, text): idx for idx, text in df["text"].items() if pd.notnull(text)}
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]
            try:
                df.at[idx, "remove_emoji"] = future.result()
            except Exception as e:
                print(f"Error removing emojis for index {idx}: {e}")
                df.at[idx, "remove_emoji"] = df.at[idx, "text"]  # Keep original text if error occurs

    return df

In [19]:
kanye_comment_processed = remove_emojis_parallel(kanye_comment)
manson_comment_processed = remove_emojis_parallel(manson_comment)
kelly_comment_processed = remove_emojis_parallel(kelly_comment)
seungri_comment_processed = remove_emojis_parallel(seungri_comment)

100%|██████████| 6362/6362 [00:00<00:00, 143018.04it/s]


In [20]:
# remove empty or whitespace only
kanye_comment_processed = kanye_comment_processed[kanye_comment_processed["remove_emoji"].str.strip().astype(bool)]
manson_comment_processed = manson_comment_processed[manson_comment_processed["remove_emoji"].str.strip().astype(bool)]
kelly_comment_processed = kelly_comment_processed[kelly_comment_processed["remove_emoji"].str.strip().astype(bool)]
seungri_comment_processed = seungri_comment_processed[seungri_comment_processed["remove_emoji"].str.strip().astype(bool)]

In [ ]:
kanye_comment_processed.head()

,text,updateDt,video_id,remove_emoji
123226,Thank you for being a leader 🙌 Stand alone sta...,2022-10-27 00:00:00+00:00,SYtkPnRK294,Thank you for being a leader Stand alone stan...
36341,pull up in the sri lanka 😭,2024-03-09 00:00:00+00:00,Z9gkv2XVXuc,pull up in the sri lanka
16042,"You know what they say, if they trying to make...",2024-04-29 00:00:00+00:00,Pmek6CSrTww,"You know what they say, if they trying to make..."


## translate

In [22]:
def detect_and_translate(text):
    """
    Detect the language of the text and translate it.
    """
    response = libre_translate(text)

    if "error" in response:
        return "error", text

    # Extract the detected language and translated text from the response
    detected_language = response["detectedLanguage"]["language"]
    translated_text = response["translatedText"]

    return detected_language, translated_text

def detect_and_translate_parallel(df):
    """
    Detect languages and translate text in parallel.
    """
    df["source"] = pd.NA
    df["translated"] = pd.NA

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(detect_and_translate, text): idx for idx, text in df["remove_emoji"].items() if pd.notnull(text)}
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]
            try:
                detected_language, translated_text = future.result()
                df.at[idx, "source"] = detected_language
                df.at[idx, "translated"] = translated_text
            except Exception as e:
                print(f"Error processing future: {e}")
                df.at[idx, "source"] = "error"
                df.at[idx, "translated"] = df.at[idx, "remove_emoji"]

    return df

In [23]:
kanye_comment_processed = detect_and_translate_parallel(kanye_comment_processed)
manson_comment_processed = detect_and_translate_parallel(manson_comment_processed)
kelly_comment_processed = detect_and_translate_parallel(kelly_comment_processed)
seungri_comment_processed = detect_and_translate_parallel(seungri_comment_processed)

100%|██████████| 6235/6235 [10:33<00:00,  9.84it/s]


In [ ]:
kanye_comment_processed.head()

,text,updateDt,video_id,remove_emoji,source,translated
163425,0:52 I feel like this would be in a alien game...,2021-09-05 00:00:00+00:00,aIhdYj4tfFo,0:52 I feel like this would be in a alien game...,en,0:52 I feel like this would be in a alien game...
126131,"It's deep!\nWake up call to all of us, start l...",2022-09-22 00:00:00+00:00,SYtkPnRK294,"It's deep!\nWake up call to all of us, start l...",en,"It's deep!\nWake up call to all of us, start l..."
129917,Baby i smiled the whole episode just to see ka...,2022-08-01 00:00:00+00:00,o6gD9_akew0,Baby i smiled the whole episode just to see ka...,en,Baby i smiled the whole episode just to see ka...
153582,Metallica black album-Kanye west,2021-11-29 00:00:00+00:00,93UpSHztaq0,Metallica black album-Kanye west,en,Metallica black album-Kanye west
181426,"""A Black Man?"" Yes sir, A Black MAN in Christ.",2019-11-09 00:00:00+00:00,ivCY3Ec4iaU,"""A Black Man?"" Yes sir, A Black MAN in Christ.",en,"""A Black Man?"" Yes sir, A Black MAN in Christ."


## stopwords

In [24]:
def remove_stopwords_parallel(df):
    """
    Remove stopwords in parallel from a DataFrame.
    """
    # Initialize the 'remove_stopword' column with NaN
    df["remove_stopword"] = pd.NA

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(remove_stopwords, text): idx for idx, text in df["translated"].items() if pd.notnull(text)}
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]
            try:
                df.at[idx, "remove_stopword"] = future.result()
            except Exception as e:
                print(f"Error removing stopwords for index {idx}: {e}")
                df.at[idx, "remove_stopword"] = df.at[idx, "translated"]  # Keep original text if error occurs

    return df

In [25]:
kanye_comment_processed = remove_stopwords_parallel(kanye_comment_processed)
manson_comment_processed = remove_stopwords_parallel(manson_comment_processed)
kelly_comment_processed = remove_stopwords_parallel(kelly_comment_processed)
seungri_comment_processed = remove_stopwords_parallel(seungri_comment_processed)

100%|██████████| 6235/6235 [00:00<00:00, 179897.26it/s]


In [ ]:
#kanye_comment_processed.head()

,text,updateDt,video_id,remove_emoji,source,translated,remove_stopword
163425,0:52 I feel like this would be in a alien game...,2021-09-05 00:00:00+00:00,aIhdYj4tfFo,0:52 I feel like this would be in a alien game...,en,0:52 I feel like this would be in a alien game...,0:52 I feel like would alien game murder horro...
126131,"It's deep!\nWake up call to all of us, start l...",2022-09-22 00:00:00+00:00,SYtkPnRK294,"It's deep!\nWake up call to all of us, start l...",en,"It's deep!\nWake up call to all of us, start l...","It 's deep ! Wake call us , start living life ..."
129917,Baby i smiled the whole episode just to see ka...,2022-08-01 00:00:00+00:00,o6gD9_akew0,Baby i smiled the whole episode just to see ka...,en,Baby i smiled the whole episode just to see ka...,Baby smiled whole episode see kanye time today
153582,Metallica black album-Kanye west,2021-11-29 00:00:00+00:00,93UpSHztaq0,Metallica black album-Kanye west,en,Metallica black album-Kanye west,Metallica black album-Kanye west
181426,"""A Black Man?"" Yes sir, A Black MAN in Christ.",2019-11-09 00:00:00+00:00,ivCY3Ec4iaU,"""A Black Man?"" Yes sir, A Black MAN in Christ.",en,"""A Black Man?"" Yes sir, A Black MAN in Christ.","`` A Black Man ? '' Yes sir , A Black MAN Chri..."


## lemmatize

In [26]:
def lemmatize_parallel(df):
    """
    Lemmatize text in parallel from a DataFrame.
    """
    # Initialize the 'lemmatization' column with NaN
    df["lemmatization"] = pd.NA

    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(get_lemma, text): idx for idx, text in df["remove_stopword"].items() if pd.notnull(text)}
        for future in tqdm(as_completed(futures), total=len(futures)):
            idx = futures[future]
            try:
                df.at[idx, "lemmatization"] = future.result()
            except Exception as e:
                print(f"Error lemmatizing text for index {idx}: {e}")
                df.at[idx, "lemmatization"] = df.at[idx, "remove_stopword"]  # Keep original text if error occurs

    return df

In [27]:
kanye_comment_processed = lemmatize_parallel(kanye_comment_processed)
manson_comment_processed = lemmatize_parallel(manson_comment_processed)
kelly_comment_processed = lemmatize_parallel(kelly_comment_processed)
seungri_comment_processed = lemmatize_parallel(seungri_comment_processed)

100%|██████████| 6235/6235 [00:38<00:00, 161.53it/s]


In [ ]:
kanye_comment_processed.head()

,text,updateDt,video_id,remove_emoji,source,translated,remove_stopword,lemmatization
163425,0:52 I feel like this would be in a alien game...,2021-09-05 00:00:00+00:00,aIhdYj4tfFo,0:52 I feel like this would be in a alien game...,en,0:52 I feel like this would be in a alien game...,0:52 I feel like would alien game murder horro...,0:52 I feel like would alien game murder horro...
126131,"It's deep!\nWake up call to all of us, start l...",2022-09-22 00:00:00+00:00,SYtkPnRK294,"It's deep!\nWake up call to all of us, start l...",en,"It's deep!\nWake up call to all of us, start l...","It 's deep ! Wake call us , start living life ...","it be deep ! wake call we , start live life wa..."
129917,Baby i smiled the whole episode just to see ka...,2022-08-01 00:00:00+00:00,o6gD9_akew0,Baby i smiled the whole episode just to see ka...,en,Baby i smiled the whole episode just to see ka...,Baby smiled whole episode see kanye time today,Baby smile whole episode see kanye time today
153582,Metallica black album-Kanye west,2021-11-29 00:00:00+00:00,93UpSHztaq0,Metallica black album-Kanye west,en,Metallica black album-Kanye west,Metallica black album-Kanye west,Metallica black album - Kanye west
181426,"""A Black Man?"" Yes sir, A Black MAN in Christ.",2019-11-09 00:00:00+00:00,ivCY3Ec4iaU,"""A Black Man?"" Yes sir, A Black MAN in Christ.",en,"""A Black Man?"" Yes sir, A Black MAN in Christ.","`` A Black Man ? '' Yes sir , A Black MAN Chri...","` ` a Black Man ? '' yes sir , a black MAN Chr..."


## saving

In [28]:
# remove empty or whitespace only
kanye_comment_processed = kanye_comment_processed[kanye_comment_processed["lemmatization"].str.strip().astype(bool)]
manson_comment_processed = manson_comment_processed[manson_comment_processed["lemmatization"].str.strip().astype(bool)]
kelly_comment_processed = kelly_comment_processed[kelly_comment_processed["lemmatization"].str.strip().astype(bool)]
seungri_comment_processed = seungri_comment_processed[seungri_comment_processed["lemmatization"].str.strip().astype(bool)]

In [29]:
# because we changed data
kanye_comment_processed = balance_dataset(kanye_comment_processed, "kanye_west")
manson_comment_processed = balance_dataset(manson_comment_processed, "marilyn_manson")
kelly_comment_processed = balance_dataset(kelly_comment_processed, "r_kelly")
seungri_comment_processed = balance_dataset(seungri_comment_processed, "seungri")

r_kelly's canceled date: 2021-09-27 00:00:00+00:00
r_kelly's Before canceled date data: 9873, After canceled date data: 9664
r_kelly's Before canceled date data: 9664, After canceled date data: 9664

seungri's canceled date: 2019-01-31 00:00:00+00:00
seungri's Before canceled date data: 3103, After canceled date data: 3115
seungri's Before canceled date data: 3103, After canceled date data: 3103



In [30]:
kanye_comment_processed.to_csv(os.path.join(PROCESSED_DATA_PATH, "kanye_west_youtube_comments_processed.csv"), index=False)
manson_comment_processed.to_csv(os.path.join(PROCESSED_DATA_PATH, "marilyn_manson_youtube_comments_processed.csv"), index=False)
kelly_comment_processed.to_csv(os.path.join(PROCESSED_DATA_PATH, "r_kelly_youtube_comments_processed.csv"), index=False)
seungri_comment_processed.to_csv(os.path.join(PROCESSED_DATA_PATH, "seungri_youtube_comments_processed.csv"), index=False)

# t-test

In [32]:
kanye_comment_processed = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "kanye_west_youtube_comments_processed.csv"))
manson_comment_processed = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "marilyn_manson_youtube_comments_processed.csv"))
kelly_comment_processed = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "r_kelly_youtube_comments_processed.csv"))
seungri_comment_processed = pd.read_csv(os.path.join(PROCESSED_DATA_PATH, "seungri_youtube_comments_processed.csv"))

In [33]:
kanye_comment_processed.head()

,text,updateDt,video_id,remove_emoji,source,translated,remove_stopword,lemmatization
0,This is cute. Kanye I'm proud of you. Donda is...,2022-05-11 00:00:00+00:00,401hZy6Hipw,This is cute. Kanye I'm proud of you. Donda is...,en,This is cute. Kanye I'm proud of you. Donda is...,This cute . Kanye I 'm proud . Donda proud . T...,this cute . Kanye I ' m proud . donda proud . ...
1,Lil Durk carried,2021-09-02 00:00:00+00:00,txion5seTBA,Lil Durk carried,en,Lil Durk carried,Lil Durk carried,Lil Durk carry
2,uma das melhores do album,2021-11-07 00:00:00+00:00,uZET6hpfV-4,uma das melhores do album,pt,one of the best album,one best album,one good album
3,Some of y'all need to listen to Vica Versa by ...,2021-09-03 00:00:00+00:00,UArRcQEgxp8,Some of y'all need to listen to Vica Versa by ...,en,Some of y'all need to listen to Vica Versa by ...,Some y'all need listen Vica Versa Pastor Troy,some you all need listen Vica Versa Pastor Troy
4,MASTERPIECE 🏆🔥,2020-11-13 00:00:00+00:00,f6vg4ZVyUW8,MASTERPIECE,en,MASTERPIECE,MASTERPIECE,MASTERPIECE


In [38]:
kanye_comment_processed = sort_comment_by_date(kanye_comment_processed)
manson_comment_processed = sort_comment_by_date(manson_comment_processed)
kelly_comment_processed = sort_comment_by_date(kelly_comment_processed)
seungri_comment_processed = sort_comment_by_date(seungri_comment_processed)

When the sample size is very large, the critical value at a significance level of 0.05 is 1.96

In [39]:
kanye_cancellation_date = CELEBRITIES[CELEBRITIES["name"] == "kanye_west"][
    "cancellation_date"
].iloc[0]
kanye_comment_before_canceled = kanye_comment_processed[
    kanye_comment_processed["updateDt"] < kanye_cancellation_date
]
kanye_comment_after_canceled = kanye_comment_processed[
    kanye_comment_processed["updateDt"] >= kanye_cancellation_date
]
print(
    "kanye count before: {0}, count after: {1}".format(
        len(kanye_comment_before_canceled),
        len(kanye_comment_after_canceled),
    )
)

manson_cancellation_date = CELEBRITIES[CELEBRITIES["name"] == "marilyn_manson"][
    "cancellation_date"
].iloc[0]
manson_comment_before_canceled = manson_comment_processed[
    manson_comment_processed["updateDt"] < manson_cancellation_date
]
manson_comment_after_canceled = manson_comment_processed[
    manson_comment_processed["updateDt"] >= manson_cancellation_date
]
print(
    "manson count before: {0}, count after: {1}".format(
        len(manson_comment_before_canceled),
        len(manson_comment_after_canceled),
    )
)

kelly_cancellation_date = CELEBRITIES[CELEBRITIES["name"] == "r_kelly"][
    "cancellation_date"
].iloc[0]
kelly_comment_before_canceled = kelly_comment_processed[
    kelly_comment_processed["updateDt"] < kelly_cancellation_date
]
kelly_comment_after_canceled = kelly_comment_processed[
    kelly_comment_processed["updateDt"] >= kelly_cancellation_date
]
print(
    "kelly count before: {0}, count after: {1}".format(
        len(kelly_comment_before_canceled),
        len(kelly_comment_after_canceled),
    )
)

seungri_cancellation_date = CELEBRITIES[CELEBRITIES["name"] == "seungri"][
    "cancellation_date"
].iloc[0]
seungri_comment_before_canceled = seungri_comment_processed[
    seungri_comment_processed["updateDt"] < seungri_cancellation_date
]
seungri_comment_after_canceled = seungri_comment_processed[
    seungri_comment_processed["updateDt"] >= seungri_cancellation_date
]
print(
    "seungri count before: {0}, count after: {1}".format(
        len(seungri_comment_before_canceled),
        len(seungri_comment_after_canceled),
    )
)

kanye count before: 9708, count after: 9708
manson count before: 9698, count after: 9698
kelly count before: 9664, count after: 9664
seungri count before: 3103, count after: 3103


In [40]:
def sentiment_analysis_and_ttest(df_before, df_after):
    senti_analyzer = SentimentIntensityAnalyzer()
    df = pd.DataFrame()
    df2 = pd.DataFrame()

    for i, text in enumerate(df_before["lemmatization"]):
        text = str(text)
        senti_scores = senti_analyzer.polarity_scores(text)
        df = pd.concat([df, pd.DataFrame(senti_scores, index=[i])], axis=0)

    for i, text in enumerate(df_after["lemmatization"]):
        text = str(text)
        senti_scores = senti_analyzer.polarity_scores(text)
        df2 = pd.concat([df2, pd.DataFrame(senti_scores, index=[i])], axis=0)

    print("before canceled mean compound score: {:.3f}".format(df["compound"].mean()))
    print("after canceled mean compound score: {:.3f}".format(df2["compound"].mean()))

    t, p = scipy.stats.ttest_ind(df["compound"], df2["compound"])
    print("t score: {0:.3f}, p-value: {1:.3f}".format(t, p))

    return

In [41]:
print("kanye west sentiment anaylsis and ttest")
sentiment_analysis_and_ttest(
    kanye_comment_before_canceled, kanye_comment_after_canceled
)

kanye west sentiment anaylsis and ttest
before canceled mean compound score: 0.173
after canceled mean compound score: 0.114
t score: 10.047, p-value: 0.000


In [42]:
print("marilyn manson sentiment anaylsis and ttest")
sentiment_analysis_and_ttest(
    manson_comment_before_canceled, manson_comment_after_canceled
)

marilyn manson sentiment anaylsis and ttest
before canceled mean compound score: 0.235
after canceled mean compound score: 0.177
t score: 8.572, p-value: 0.000


In [43]:
print("r.kelly sentiment anaylsis and ttest")
sentiment_analysis_and_ttest(
    kelly_comment_before_canceled, kelly_comment_after_canceled
)

r.kelly sentiment anaylsis and ttest
before canceled mean compound score: 0.085
after canceled mean compound score: 0.135
t score: -7.865, p-value: 0.000


In [44]:
print("seungri sentiment anaylsis and ttest")
sentiment_analysis_and_ttest(
    seungri_comment_before_canceled, seungri_comment_after_canceled
)

seungri sentiment anaylsis and ttest
before canceled mean compound score: 0.296
after canceled mean compound score: 0.087
t score: 19.166, p-value: 0.000
